In [1]:
from datatable import (dt, f, by, ifelse, update, sort,
                       count, min, max, mean, sum, rowsum)
import pandas as pd
import numpy as np
import datetime
 
from pathlib import Path
import os

import classes

In [2]:
#build classes and attribute dictionary to interrogate bigTable
classes_field_dict = {}
for k, v in classes.__dict__.items(): 
    if  ('classes.' in str(v)) and (str(k).startswith('__') == False):
        classes_field_dict[k] = [f'_{att}' for att in v.__dict__.keys() if 
                                 str(att).startswith('__') == False
                                and type(v.__dict__[att]) == property
                                and str(att) != 'events' ]
        if (str(k) in ['Event', 'Patient']) == False:
            classes_field_dict[k].extend(classes_field_dict['Event'])
        
        if (str(k) in ['Patient']) == False:
            classes_field_dict[k].extend(['_patient_id', '_p_id', '_age', '_gender'])
        classes_field_dict[k] = list(set(classes_field_dict[k]))
    
classes_field_dict

{'Event': ['_end_date',
  '_event_type',
  '_p_id',
  '_event_value',
  '_health_dep',
  '_patient_id',
  '_start_date',
  '_age',
  '_gender'],
 'Vs': ['_end_date',
  '_event_type',
  '_p_id',
  '_event_value',
  '_health_dep',
  '_patient_id',
  '_hour',
  '_start_date',
  '_age',
  '_gender',
  '_vs_name'],
 'Diagnosis': ['_end_date',
  '_event_type',
  '_p_id',
  '_desc',
  '_event_value',
  '_health_dep',
  '_patient_id',
  '_start_date',
  '_age',
  '_gender'],
 'Procedure': ['_end_date',
  '_event_type',
  '_p_id',
  '_desc',
  '_event_value',
  '_health_dep',
  '_patient_id',
  '_start_date',
  '_age',
  '_gender'],
 'Medication': ['_dose',
  '_end_date',
  '_event_type',
  '_age',
  '_p_id',
  '_event_value',
  '_health_dep',
  '_freq',
  '_patient_id',
  '_atc',
  '_start_date',
  '_route',
  '_unit',
  '_gender'],
 'Oxigen': ['_end_date',
  '_age',
  '_event_type',
  '_p_id',
  '_event_value',
  '_health_dep',
  '_patient_id',
  '_start_date',
  '_method',
  '_gender'],
 'La

In [3]:
csv_dir = Path('out_EDA/csv')
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

str(csv_dir)

'out_EDA/csv'

In [4]:
DT = dt.Frame("out_EDA/bigTable.jay")
#dt.unique(DT[:,f._event_type]).to_csv('event_types.csv')
event_types_classes = dt.Frame("event_types_classes.csv")


In [5]:
df = DT.to_pandas()
df._start_date = pd.to_datetime(df._start_date) 

In [6]:
df._end_date = pd.to_datetime(df._end_date, errors = 'ignore') 

In [7]:
df['_p_id'] = None
patients_df = df[classes_field_dict['Patient']].drop_duplicates().reset_index()

In [8]:
#profile each class
profile = False
if profile:
    from pandas_profiling import ProfileReport
    profile = ProfileReport(patients_df,  pool_size = 32)            
    profile.to_file(f'out_EDA/Patient.html')

    #profile = ProfileReport(df, minimal=True, pool_size = 32)
    for i,r in event_types_classes.to_pandas().iterrows():
        event_type = r._event_type
        fields = classes_field_dict[r['class']]
        print(event_type, fields)
        df_ = DT[(f._event_type == event_type),fields].to_pandas()
        print(f'Starting to profile: {event_type} size {df.shape}')
        try:
            profile = ProfileReport(df_,  pool_size = 32)
            profile.to_file(f'out_EDA/{event_type}.html')
        except: 
            print(f'skipped {event_type}')
   

In [9]:
patients_df['_p_id'] = patients_df.index
p_dict = patients_df[[ '_patient_id', '_p_id']].set_index('_patient_id').to_dict()['_p_id']


In [10]:
#compress patient id to int in field _p_id
def f(x):
    i = p_dict[x]
    
    return i

df['_p_id'] = df._patient_id.apply(lambda x: f(x))


In [11]:
#load events of different types in a dictionary of df (df_dict)
df_dict = {}
for i,r in event_types_classes.to_pandas().iterrows():
        event_type = r._event_type
        fields = classes_field_dict[r['class']]
        df_dict[event_type] = df.loc[df._event_type == event_type, fields]

In [12]:
df_dict.keys()

dict_keys(['Diagnosis', 'Fluid', 'Lab', 'Lab Sars-cov-2', 'Medication', 'Oxigen', 'Procedure', 'Scale', 'State Admission', 'State Bed', 'State Discharge', 'Text 1DX', 'Text 2DX', 'Text CX', 'Text Discharge', 'Text EV', 'Text MH', 'Text PE', 'Text PLAN', 'Text PX', 'Text QX', 'Text RE', 'Text TX', 'Text sDX', 'VS', "['CR'] ['CHEST']", "['CR'] ['Pecho']", "['CR'] ['SKULL']", "['CR'] ['THORAX']", "['CR'] nan", "['CT'] ['ABDOMEN']", "['CT'] ['CHEST']", "['CT'] ['T A C A R']", "['CT'] ['TAP']", "['CT'] ['TEP']", "['CT'] ['TOR ABD PELVICO']", "['CT'] ['TORAX']", "['CT'] ['TXABDOMENPELVIS']", "['CT'] ['TXABDPELV']", "['CT'] ['TXABDPELVIS']", "['CT'] nan", "['DX'] ['ABDOMEN']", "['DX'] ['CHEST']", "['DX'] ['FOOT']", "['DX'] ['PELVIS']", "['DX'] ['PORT CHEST']", "['DX'] ['RIB']", "['DX'] ['T?RAX']", "['DX'] ['THORAX']", "['DX'] ['TORAX']", "['DX'] ['TÒRAX']"])

In [13]:
dx = pd.DataFrame(df_dict['Diagnosis'][['_event_value', '_desc']].value_counts())
dx.to_csv('Terminology/icd_dx_counts.tsv')

In [14]:
covid_rel_dx = pd.read_csv('Terminology/icd_covid_dx_counts.csv') #after manual selection of covid related dx

l_covid_dx = covid_rel_dx[covid_rel_dx.covid_related_clinic_dx == 'x']._event_value.values


In [15]:
l_covid_image = ['C0277877', 'C2073538', 'C3544344', 'C2073672','C2073583', 'C1332240', 
                   'C0521530', 'C3669021', 'C3669021', 
                   'C1443940', 'C0032285', 'C1412002', 'C0032310', 'C5203670', 'C5203671']

In [16]:
cov_df =  df_dict['Lab Sars-cov-2']
cov_df[cov_df._test_name== 'PCR']._event_value.unique()

array(['POSITIVO', 'NEGATIVO', 'INDETERMINADO'], dtype=object)

In [17]:
image_event_types = ["['CR'] ['CHEST']", "['CR'] ['Pecho']", "['CR'] ['SKULL']", "['CR'] ['THORAX']", "['CR'] nan", "['CT'] ['ABDOMEN']", "['CT'] ['CHEST']", "['CT'] ['T A C A R']", "['CT'] ['TAP']", "['CT'] ['TEP']", "['CT'] ['TOR ABD PELVICO']", "['CT'] ['TORAX']", "['CT'] ['TXABDOMENPELVIS']", "['CT'] ['TXABDPELV']", "['CT'] ['TXABDPELVIS']", "['CT'] nan", "['DX'] ['ABDOMEN']", "['DX'] ['CHEST']", "['DX'] ['FOOT']", "['DX'] ['PELVIS']", "['DX'] ['PORT CHEST']", "['DX'] ['RIB']", "['DX'] ['T?RAX']", "['DX'] ['THORAX']", "['DX'] ['TORAX']", "['DX'] ['TÒRAX']"]

In [18]:

def covid_cui(x):
    if not pd.isna(x): 
        x = x.replace('[','').replace(']','').split(',')
        covid_cuis = [str(cui)  for cui in x if cui in l_covid_image]
        result = covid_cuis if len(covid_cuis) > 0 else False
        
    return result
df['covid_image'] = False
df.loc[df._event_type.isin(image_event_types),['covid_image']] = df[df._event_type.isin(image_event_types)]._cui_list.apply(lambda x: covid_cui(x))


In [19]:
x = df
discharge = x[((x._event_type == 'State Discharge') )]._event_value
discharge.unique()

array(['Éxitus', 'Equipo atención primaria',
       'Traslado Residencia o Centro Socio-Sanitario asistido',
       'Consultas externas', 'Traslado Hospital de agudos', 'Domicilio',
       'Otros', 'Unidad de Hospitalización a domicilio',
       'Traslado Hospital Media larga estancia', 'Hospital de Día',
       'Alta voluntaria', 'Fuga', 'In extremis'], dtype=object)

In [20]:

ox = x[(x._event_type == 'Oxigen')]._method
for m in ox.unique():
    print(m)
    print(x[(x._event_type == 'Oxigen') & (x._method == m)]._event_value.value_counts())


Mascarilla
50.0     133
35.0     113
40.0      96
30.0      85
28.0      80
24.0      36
21.0      14
100.0      6
Name: _event_value, dtype: int64
Gafas nasales
nan      2315
24.0       90
28.0       73
21.0       54
50.0       36
30.0       22
100.0      18
40.0       17
35.0        8
Name: _event_value, dtype: int64
Mascarilla con reservorio
nan      57
100.0    32
50.0     24
35.0      4
28.0      2
40.0      1
Name: _event_value, dtype: int64
nan
Series([], Name: _event_value, dtype: int64)
nan
50.0     19
24.0     15
21.0     11
100.0    10
30.0      5
35.0      3
40.0      1
28.0      1
Name: _event_value, dtype: int64
Cánula transtraqueal
24.0    5
28.0    3
30.0    1
50.0    1
Name: _event_value, dtype: int64


In [21]:
oxygen_dic = pd.read_csv('Terminology/umls_oxygen_techniques.tsv', index_col = 0, header = None).to_dict('index')
oxygen_dic_rev = {sk : k.split('\t')[0] for k in oxygen_dic.keys() for sk in k.lower().split('\t')[1:] }

oxygen_dic = { k.split('\t')[0]:  k.lower().split('\t')[1:] for k in oxygen_dic.keys()  }
oxygen_dic

{'C0740087': ['oxygen nasal cannula', 'gafas nasales', 'gafas'],
 'C0181744': ['oxygen mask', 'mascarilla simple', 'mascarilla'],
 'C1960097': ['high flow oxygen nasal cannula', 'canulas nasales alto flujo '],
 'C0181748': ['venturi oxygen face mask',
  'mascarilla tipo venturi',
  'vmx',
  'ventimask',
  'masc. o2 vmk'],
 'C0180824': ['face tent oxygen delivery device'],
 'C0182123': ['oxygen concentrator (device)',
  'concentrador de oxigeno',
  'mascara reservorio',
  'mascarilla con reservorio',
  'reservorio',
  'masc. o2 reservorio'],
 'C1960094': ['oxygen ventilator', 'ventilacion mecanica', 'vm', 'respirador'],
 'C0419003': ['mechanically assisted spontaneous ventilation',
  'cpap',
  'bipap'],
 'C0021925': ['intubation (procedure)',
  'intubacion',
  'ventilacion mecanica invasiva',
  'iot'],
 'C0877798': ['transtracheal oxygen catheter',
  'intubación transtraqueal',
  'ventilacion mecanica invasiva transtraqueal',
  'canula transtraqueal'],
 'C0015357': ['extracorporeal memb

In [22]:
oxigen_low = ['C0740087', 'C0181744']
oxigen_high = ['C1960097', 'C0180824', 'C0182123', 'C1960094', 'C0419003', 'C0021925', 'C0877798', 'C0015357'] #note: non-invasive mechanical ventilation (C0419003) is included as high flow as a simplification to help with 'who' outcome scale calculations, VMX is excluded (it requires fiO2 > 50% to be considered high flow device)
oxigen_pot_high = ['C1960097', 'C0181748' , 'C0180824', 'C0182123', 'C1960094', 'C0419003', 'C0021925', 'C0877798', 'C0015357'] #note: non-invasive mechanical ventilation (C0419003) is included as high flow as a simplification to help with 'who' outcome scale calculations 

In [23]:
df['oxigen_method'] = np.nan

In [24]:
import unidecode
df.loc[df._event_type== 'Oxigen',['oxigen_method']] = df[df._event_type== 'Oxigen']._method.apply(lambda x: oxygen_dic_rev[unidecode.unidecode(str(x)).lower()] if str(x).lower() in oxygen_dic_rev else np.nan )
df.oxigen_method.value_counts()

C0740087    2633
C0181744     563
C0182123     120
Name: oxigen_method, dtype: int64

In [25]:
df.loc[(df._event_type== 'VS')  & (x._vs_name.str.lower().str.contains(
    '(metodo de administra|tipo oxigeno|oxigenoterapia|o2 suplem)', regex = True)),['oxigen_method']] = df[
    (df._event_type== 'VS')  & (x._vs_name.str.lower().str.contains(
    '(metodo de administra|tipo oxigeno|oxigenoterapia|o2 suplem)', regex = True))]._event_value.apply(lambda x: oxygen_dic_rev[unidecode.unidecode(str(x)).lower()] if str(x).lower() in oxygen_dic_rev else np.nan  )
df.oxigen_method.value_counts()

/home/auri/anaconda3/envs/nlpMED/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


C0740087    4433
C0181748     832
C0181744     563
C0182123     394
C0419003       7
Name: oxigen_method, dtype: int64

In [26]:

ox_vs = x[((x._event_type == 'VS')  & (x._vs_name.str.lower().str.contains(
    '(o2|oxig)', regex = True)))]._vs_name
for o in ox_vs.unique():
    print(o)
    print(x[((x._event_type == 'VS')  & (x._vs_name == o))]._event_value.value_counts())

Volumen de oxigeno
2.0      7091
3.0      3732
1.0      2809
4.0      2618
15.0     2220
6.0      1136
8.0       972
1.5       930
10.0      692
5.0       567
2.5       383
12.0      358
0.5       335
3.5       169
50.0      136
7.0       103
9.0        87
nan        81
40.0       58
28.0       43
30.0       42
35.0       37
25.0       26
45.0       24
11.0       21
24.0       15
32.0       15
0.0        15
13.0       15
4.5        14
31.0       13
20.0       13
36.0       13
14.0        9
16.0        6
7.5         5
100.0       2
21.0        2
0.25        2
39.0        1
75.0        1
34.0        1
33.0        1
5.5         1
36.2        1
17.0        1
3.4         1
44.0        1
8.5         1
95.0        1
98.0        1
48.0        1
0.1         1
Name: _event_value, dtype: int64
Saturacion O2
97.0    16013
96.0    15683
98.0    15571
95.0    12467
94.0     9145
        ...  
29.0        1
33.8        1
35.5        1
38.6        1
48.0        1
Name: _event_value, Length: 105, dtype

In [27]:
df['fiO2'] = np.nan
df.loc[df._event_type== 'Oxigen',['fiO2']] = df[df._event_type== 'Oxigen']._event_value.apply(lambda x: float(x))
df.fiO2.describe()

count    1149.000000
mean       37.994778
std        18.741548
min        21.000000
25%        28.000000
50%        30.000000
75%        50.000000
max       100.000000
Name: fiO2, dtype: float64

In [28]:
df.loc[(df._event_type== 'VS')& (df._vs_name == 'Volumen de oxigeno'),['fiO2']] = df[(df._event_type== 'VS')& (df._vs_name == 'Volumen de oxigeno')]._event_value.apply(lambda x: 21 + (float(x) * 4) if float(x) <21 else float(x))
df.fiO2.describe() #fiO2 should be >= 21% - 100%

count    25891.000000
mean        38.477123
std         16.517411
min         21.000000
25%         29.000000
50%         33.000000
75%         41.000000
max        101.000000
Name: fiO2, dtype: float64

In [29]:
df.loc[(df._event_type== 'VS')& (df._vs_name == 'Oxigeno'),['fiO2']] = df[(df._event_type== 'VS')& (df._vs_name == 'Oxigeno')]._event_value.apply(lambda x: 21 if str(x).startswith('Sin ') else np.nan )
df.fiO2.describe() #if patient is reported as "Sin Oxigeno"  then FiO2 = 21%

count    53865.000000
mean        29.400635
std         14.400844
min         21.000000
25%         21.000000
50%         21.000000
75%         33.000000
max        101.000000
Name: fiO2, dtype: float64

In [30]:
df.loc[(df._event_type== 'VS')& (df._vs_name == 'O2 Suplementario'),['fiO2']] = df[(df._event_type== 'VS')& (df._vs_name == 'O2 Suplementario')]._event_value.apply(lambda x: 21 if str(x) == 'NO' else np.nan )
df.fiO2.describe() #if patient is reported as "No" in "O2 Suplementario" then FiO2 = 21%

count    54471.000000
mean        29.307176
std         14.347594
min         21.000000
25%         21.000000
50%         21.000000
75%         31.000000
max        101.000000
Name: fiO2, dtype: float64

In [31]:
df.loc[(df._event_type== 'VS')& (df._vs_name.str.startswith('FiO2')),['fiO2']] = df[(df._event_type== 'VS')&(df._vs_name.str.startswith('FiO2'))]._event_value.apply(lambda x: float(x) if float(x) else np.nan)  
df.fiO2.describe() #add reported FiO2

count    54981.000000
mean        29.418057
std         14.346494
min         21.000000
25%         21.000000
50%         21.000000
75%         33.000000
max        101.000000
Name: fiO2, dtype: float64

In [32]:
df[pd.isna(df.fiO2) == False]._p_id.nunique()

3650

In [33]:
df[pd.isna(df.oxigen_method) == False]._p_id.nunique()

1409

In [34]:
df[pd.isna(df.fiO2) == False].groupby(['_health_dep']).count()

,_start_date,_end_date,_event_type,_event_value,_test_name,_unit,_limits,_norm_value,_patient_id,_age,...,_dose,_freq,_route,_atc,_desc,_scale_name,_p_id,covid_image,oxigen_method,fiO2
_health_dep,,,,,,,,,,,,,,,,,,,,,
01,3902,2,3902,3902,0,0,0,0,3902,3902,...,0,0,0,0,0,0,3902,3902,5,3902
02,9051,288,9051,9051,0,0,0,0,9051,9051,...,0,0,0,0,0,0,9051,9051,362,9051
05,17117,0,17117,17117,0,0,0,0,17117,17117,...,0,0,0,0,0,0,17117,17117,9,17117
06,2853,97,2853,2853,0,0,0,0,2853,2853,...,0,0,0,0,0,0,2853,2853,145,2853
08,4283,2,4283,4283,0,0,0,0,4283,4283,...,0,0,0,0,0,0,4283,4283,20,4283
12,757,56,757,757,0,0,0,0,757,757,...,0,0,0,0,0,0,757,757,83,757
16,14,0,14,14,0,0,0,0,14,14,...,0,0,0,0,0,0,14,14,0,14
17,2004,199,2004,2004,0,0,0,0,2004,2004,...,0,0,0,0,0,0,2004,2004,268,2004
18,4393,0,4393,4393,0,0,0,0,4393,4393,...,0,0,0,0,0,0,4393,4393,9,4393


In [35]:
print(x[(x._event_type == 'State Discharge')]._event_value.unique())
discharge_dict = {}


['Éxitus' 'Equipo atención primaria'
 'Traslado Residencia o Centro Socio-Sanitario asistido'
 'Consultas externas' 'Traslado Hospital de agudos' 'Domicilio' 'Otros'
 'Unidad de Hospitalización a domicilio'
 'Traslado Hospital Media larga estancia' 'Hospital de Día'
 'Alta voluntaria' 'Fuga' 'In extremis']


In [36]:
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'BARTHEL')]._event_value.unique()

array(['100/100 - DEPENDIENTE LEVE', '0/100 - DEPENDENCIA TOTAL',
       '70/100 - DEPENDIENTE LEVE', '90/100 - DEPENDIENTE LEVE',
       '55/100 - DEPENDIENTE MODERADO', '75/100 - DEPENDIENTE LEVE',
       '20/100 - DEPENDIENTE GRAVE', '65/100 - DEPENDIENTE LEVE',
       '5/100 - DEPENDENCIA TOTAL', '95/100 - DEPENDIENTE LEVE',
       '30/100 - DEPENDIENTE GRAVE', '35/100 - DEPENDIENTE GRAVE',
       '80/100 - DEPENDIENTE LEVE', '45/100 - DEPENDIENTE MODERADO',
       '50/100 - DEPENDIENTE MODERADO', '25/100 - DEPENDIENTE GRAVE',
       '15/100 - DEPENDENCIA TOTAL', '85/100 - DEPENDIENTE LEVE',
       '10/100 - DEPENDENCIA TOTAL', '40/100 - DEPENDIENTE MODERADO',
       '60/100 - DEPENDIENTE LEVE', '100/100 - INDEPENDIENTE'],
      dtype=object)

In [37]:
df['barthel'] = np.nan
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'BARTHEL'), ['barthel']] = df[
    (df._event_type == 'Scale') &  (df._scale_name == 'BARTHEL')]._event_value.apply(lambda x: int(str(x).split('/')[0]) if not pd.isna(x) else np.na)
df[pd.isna(df.barthel) == False]._p_id.nunique()

2256

In [38]:
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'SOFA')]._event_value.unique()

array(['4/20 - LEVE', '1/20 - LEVE', '2/20 - LEVE',
       '0/24 - MORTALIDAD < 10%', '2/24 - MORTALIDAD < 10%',
       '3/24 - MORTALIDAD 10-20%', '1/24 - MORTALIDAD < 10%',
       '6/24 - MORTALIDAD > 20%', '4/24 - MORTALIDAD 10-20%',
       '2/24 - MENOR 10% DE MORTALIDAD'], dtype=object)

In [39]:
df['sofa'] = np.nan
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'SOFA'), ['sofa']] = df[
    (df._event_type == 'Scale') &  (df._scale_name == 'SOFA')]._event_value.apply(lambda x: int(str(x).split('/')[0]) if not pd.isna(x) else np.na)
df[pd.isna(df.sofa) == False]._p_id.nunique()

69

In [40]:
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'GLASGOW')]._event_value.unique()

array(['10/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '12/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '3/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '13/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '11/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '7/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '8/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '6/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '15/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '10/15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '12/15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '14/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '5/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '9/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '4/15 - A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '15/15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '12/15 - /15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '4/15 - /15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '8/15 - /15 A MENOR PUNTUACIÓN, MÁS PROBLEMAS',
       '15/15 -

In [41]:
df['glasgow'] = np.nan
df.loc[(df._event_type == 'Scale') &  (df._scale_name == 'GLASGOW'), ['glasgow']] = df[
    (df._event_type == 'Scale') &  (df._scale_name == 'GLASGOW')]._event_value.apply(lambda x: int(str(x).split('/')[0]) if not pd.isna(x) else np.na)
df[pd.isna(df.glasgow) == False]._p_id.nunique()

100

In [42]:
uci_patients = x[((x._event_type == 'State Bed') & (x._event_value == 'UCI') )]._p_id.unique()

In [43]:
def who_out_scale(x, eval_date):
    in_date = x.iloc[0]._start_date
    
    out_date = x.iloc[-1]._start_date
    verbose = False
    if verbose:
        print(in_date, out_date)
        print(f'date outcome evaluation: {eval_date}')
        print(x[pd.isna(x.fiO2) == False][['_start_date', '_event_value', 'fiO2']])
        print(x.oxigen_method.unique())
    
    #Scale- the WHO COVID Outcomes Scale: Used as primary efficacy outcome of treatments. The primary outcome defined as clinical status 14 days after admission 
    #assessed with a 7-category ordinal scale  recommended by the World Health Organization (at baseline, 7 and at 14 days or discharge -whatever occurs first): 
    #https://www.tandfonline.com/doi/full/10.1080/19466315.2020.1811148 
    #Definition of category in Cao et al. (2020)
    #7, death; 
    #6, hospitalized, receiving extracorporeal membrane oxygenation (ECMO) or invasive mechanical ventilation; 
    #5, hospitalized, receiving noninvasive mechanical ventilation or nasal high-flow oxygen therapy; 
    #4, hospitalized, receiving supplemental oxygen without positive pressure or high flow; 
    #3, hospitalized, not receiving supplemental oxygen; 
    #2, not hospitalized and unable to perform normal activities; 
    #1, not hospitalized and able to perform normal activities. 
    
    
    
     #ICU 
    date_icu_in = x[((x._event_type == 'State Bed') & (x._event_value == 'UCI') & (x._start_date >= in_date ) & (x._start_date <= eval_date ))]._start_date
    date_icu_out = x[((x._event_type == 'State Bed') & (x._event_value == 'UCI') & (x._start_date >= in_date ) & (x._start_date <= eval_date ))]._end_date
    if not date_icu_in.empty:
        date_icu_in = date_icu_in.iloc[0]
    else: 
        date_icu_in = False
        
    if not date_icu_out.empty: #assume only one ICU admission, TODO improve
        date_icu_out = date_icu_out.iloc[-1]
    else: 
        date_icu_out = False
    
    
    #discharge 
    discharge = x[((x._event_type == 'State Discharge') & (x._start_date >= in_date ) & (x._start_date <= eval_date ))]
    #Fatal
    fatal = False
    date_death = None
    if not discharge[discharge._event_value.isin( ['Éxitus', 'In extremis'])].empty: 
        date_death = discharge[discharge._event_value.isin( ['Éxitus', 'In extremis'])].iloc[0]._start_date
        fatal = True 
       
    #Critic 
    critic = fatal or date_icu_in
        
    
    
    #Oxygens from in_date to outcome date (high flow = if > 50% FiO2 or (-when FiO2 is not present-) if patient has a potentially high-flow device)
    ox_high_device =  x[(x.oxigen_method.isin(oxigen_high) & (x._start_date <= eval_date ) )]._start_date 
    if not ox_high_device.empty:
        ox_high_device = ox_high_device.iloc[-1]
    else: 
        ox_high_device = False
        
    ox_high_fio2 = x[(x.fiO2 >= 50) &  (x._start_date <= eval_date )]._start_date 
    if not ox_high_fio2.empty:
        ox_high_fio2 = ox_high_fio2.iloc[-1]
    else:
        ox_high_fio2 = False
        
    ox_low_fio2 =x[(x.fiO2 < 50) & (x._start_date <= eval_date )]._start_date 
    if not ox_low_fio2.empty:
        ox_low_fio2 = ox_low_fio2.iloc[-1]
    else:
        ox_low_fio2 = False
    
    ox_low_device =  x[(x.oxigen_method.isin(oxigen_low) & (x._start_date <= eval_date ))]._start_date 
    if not ox_low_device.empty:
        ox_low_device = ox_low_device.iloc[-1]
    else :
        ox_low_device = False
    
    
    
    who_covid_out = None
    if date_death and date_death <= eval_date:
        who_covid_out = 7
    elif date_icu_in  and ((date_icu_in <= eval_date) and not date_icu_out):
        who_covid_out = 6
    elif ox_high_device and not ox_low_device : 
        who_covid_out = 5
    elif ox_high_fio2 and not ox_low_fio2:
        who_covid_out = 5
    elif (ox_high_device and  ox_low_device) and (ox_high_device >= ox_low_device ):
        who_covid_out = 5    
    elif (ox_high_fio2 and  ox_low_fio2) and (ox_high_fio2 >= ox_low_fio2):
        who_covid_out = 5
    elif discharge.empty and (ox_low_fio2 or ox_low_device):
        who_covid_out = 4
    elif discharge.empty or not discharge[discharge._event_value.isin(['Traslado Hospital de agudos'])].empty  : #still hospitalized
        who_covid_out = 3 
    elif not discharge[discharge._event_value.isin(['Traslado Residencia o Centro Socio-Sanitario asistido', 
                                      'Unidad de Hospitalización a domicilio', 'Traslado Hospital Media larga estancia' ])].empty:
        who_covid_out = 2
    else:
        who_covid_out = 1
    
    if verbose:
        print(out_date)
        print(discharge)
    return who_covid_out



In [44]:
for n_days in range(28,30):
    who_out = df[df._p_id == 0].groupby('_p_id').apply(lambda y: who_out_scale(y, y.iloc[0]._start_date +  datetime.timedelta(n_days)))
    print(f'outcome day {n_days}, WHO score {who_out.iloc[0]}')

outcome day 28, WHO score 3
outcome day 29, WHO score 7


In [45]:
#generate tables for experiments
ex1_columns = ['_patient_id','_p_id', '_health_dep','_age', '_gender', 'date_in', 'n_days_in', 'worst_barthel_scale', 'n_covid_rel_dx', 'covid_rel_dx', 'n_covid_rel_image','covid_rel_image', 'n_PCRs_+',
               'n_days_icu', 'worst_sofa_scale', 'worst_glasgow','critic', 'severity', '_discharge', 'who_out_scale']

In [46]:

#1º Select only admissions with concomitant events associated to ( ( covid related dx-cx) OR (Rx-thorax with infiltrates or condensations or pneumonia)) AND b) PCR confirmed COVID 

def table(x_p, df_exp): #x_p is a dataframe with all events for a given patient, one patient may have >= 1 admissions
    _p_id = x_p.iloc[0]._p_id
    
    #iterate admissions
    for in_date,out_date in list(zip(x_p[(x_p._event_type == 'State Admission')]._start_date, x_p[(x_p._event_type == 'State Discharge')]._start_date)):
        
        #filter events for this admission
        x = x_p[(x_p._start_date >= in_date ) & (x_p._start_date <= out_date )]
        
        #n_days_in 
        try:
            n_days_in = out_date - in_date
        except:
            print(_p_id)
            print(f'n_days_in: {n_days_in}')
            print(x)
            
        #scales (barthel best is 100, sofa worst is 24, glasgow best is 15)
        worst_barthel_scale = np.min(x[pd.isna(x.barthel) == False].barthel.values) if not x[pd.isna(x.barthel) == False].empty else np.nan
        worst_sofa_scale = np.max(x[pd.isna(x.sofa) == False].sofa.values) if not x[pd.isna(x.sofa) == False].empty else np.nan
        worst_glasgow_scale = np.min(x[pd.isna(x.glasgow) == False].glasgow.values) if not x[pd.isna(x.glasgow) == False].empty else np.nan
            
        #covid related dx-cx
        dx = x[((x._event_type == 'Diagnosis') & (x._event_value.isin(l_covid_dx)) & (x._start_date >= in_date ) & (x._start_date <= out_date ))]
        #covid related image
        rx = x[ ((x.covid_image != False) & (x._start_date >= in_date ) & (x._start_date <= out_date ))]
        #covid confirmed by PCR (extending 3 weeks before admission )
        cov = x[((x._event_type == 'Lab Sars-cov-2') & (x._event_value == 'POSITIVO') & (x._start_date >= in_date - datetime.timedelta(7*3) ) & (x._start_date <= out_date ))]
        
        #ICU 
        date_icu_in = x[((x._event_type == 'State Bed') & (x._event_value == 'UCI') & (x._start_date >= in_date ) & (x._start_date <= out_date ))]._start_date
        date_icu_out = x[((x._event_type == 'State Bed') & (x._event_value == 'UCI') & (x._start_date >= in_date ) & (x._start_date <= out_date ))]._end_date
        if date_icu_in.shape[0] > 0:
            date_icu_in = date_icu_in.iloc[0]
            date_icu_out = date_icu_out.iloc[0] #assume only one ICU admission, TODO improve
        else: 
            date_icu_in = False
            date_icu_out = False
        #discharge
        discharge = x[((x._event_type == 'State Discharge') & (x._start_date >= in_date ) & (x._start_date <= out_date ))]
        
        #Fatal
        fatal = False
        date_death = None
        death_event = discharge[discharge._event_value.isin( ['Éxitus', 'In extremis'])]
        if death_event.shape[0] > 0: 
            date_death = death_event._start_date
            fatal = True 
        
        #Critic 
        critic = fatal or date_icu_in
        
        
        
        #Severity grade (following AE grading system 1-5: 
        #1-mild(ER only, does not require hospitalization), 
        #2-moderate( hospitalization <= 5 days)
        #3-severe(prolongs hospitalization > 5 days), 
        #4-critical(life threatening = UCI), 
        #5-fatal 
        #Source: lab paper
        severity = None
        if fatal:
            severity = 5
        elif (date_icu_in ):
            severity = 4
        elif(n_days_in.days > 5 ):
            severity = 3
        elif (n_days_in.days in range(1,6)): 
            severity = 2 
        elif (n_days_in == 0 ):
            severity = 1
        
        #Scale- 4C mortality score IASRIC 
        
        
            
        row_dict = dict.fromkeys(ex1_columns)
        row_dict['_p_id'] = _p_id
        row_dict['_patient_id'] = x.iloc[0]._patient_id
        row_dict['_health_dep'] = x.iloc[0]._health_dep
        row_dict['_gender'] = x.iloc[0]._gender
        row_dict['_age'] = x.iloc[0]._age
        row_dict['date_in'] = in_date
        row_dict['n_days_in'] = n_days_in
        row_dict['worst_barthel_scale'] = worst_barthel_scale
        row_dict['n_covid_rel_image'] = rx.shape[0] #n images with related covid findings
        row_dict['covid_rel_image'] = list(set([l for i in rx.covid_image.values for l in  i]))
        row_dict['n_covid_rel_dx'] = dx.shape[0]
        row_dict['covid_rel_dx'] = dx._desc.str.strip().unique()
        row_dict['n_PCRs_+'] = cov.shape[0]
        row_dict['worst_glasgow_scale'] = worst_glasgow_scale
        row_dict['worst_sofa_scale'] = worst_sofa_scale
        row_dict['date_icu_in'] = date_icu_in
        row_dict['n_days_icu'] = (date_icu_out - date_icu_in) if date_icu_in else 0
        row_dict['_discharge'] = discharge._event_value.values[0]
        row_dict['critic'] = fatal or date_icu_in
        row_dict['severity'] = severity
        n = 14
        row_dict['who_out_scale'] = [who_out_scale(x, in_date +  datetime.timedelta(n)) for n in range(0,n) ]
        df_exp = df_exp.append(row_dict, ignore_index = True)
        
        if (((dx.shape[0] > 0) or (rx.shape[0] >  0)) and (cov.shape[0]> 0)) :
            pass
            #df_new = pd.DataFrame([_p_id], columns=ex1_columns)
            #df_exp= pd.concat([df_exp, df_new['_p_id']], ignore_index = True)
            #print(dx.shape[0],rx.shape[0],cov.shape[0])
            
    return df_exp
df_experiment = pd.DataFrame(columns = ex1_columns)
df_experiment = df.groupby('_p_id').apply(lambda x: table(x, df_experiment))
#df_experiment = df[df._p_id.isin(uci_patients)].groupby('_p_id').apply(lambda x: table(x, df_experiment))
df_experiment

,,_patient_id,_p_id,_health_dep,_age,_gender,date_in,n_days_in,worst_barthel_scale,n_covid_rel_dx,covid_rel_dx,...,n_PCRs_+,n_days_icu,worst_sofa_scale,worst_glasgow,critic,severity,_discharge,who_out_scale,date_icu_in,worst_glasgow_scale
_p_id,,,,,,,,,,,,,,,,,,,,,,
0,0,01d7a0442ab14089134c009df5aafdc1a8d206197e27db...,0,08,93,woman,2020-04-06,1 days,NaN,2,"[OTROS TIPOS DE NEUMONIA VIRICA, INSUFICIENCIA...",...,0,0,NaN,None,True,5,Éxitus,"[3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]",0,NaN
1,0,0354b2304dcf9ae94bf651001598c758b654c23315c1bb...,1,08,66,woman,2020-04-07,7 days,NaN,4,"[OTROS TIPOS DE NEUMONIA VIRICA, OTROS CORONAV...",...,0,0,NaN,None,False,3,Equipo atención primaria,"[3, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1]",0,NaN
2,0,04392047b85800a0ee5990f454c7c4b484cad8daee3383...,2,08,81,man,2020-10-04,4 days,NaN,0,[],...,4,0,NaN,None,False,2,Equipo atención primaria,"[3, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,NaN
3,0,046df9b449c71047f3c62d9fde422e61100369b7c71ceb...,3,08,90,man,2020-03-21,7 days,NaN,4,"[OTROS TIPOS DE NEUMONIA VIRICA, OTROS CORONAV...",...,1,0,NaN,None,True,5,Éxitus,"[3, 4, 4, 5, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7]",0,NaN
4,0,05abb2eafb334e14b0de26bd5aff9fe154e5e1954ab4a5...,4,08,88,woman,2020-02-12,6 days,NaN,0,[],...,0,0,NaN,None,False,3,Traslado Residencia o Centro Socio-Sanitario a...,"[3, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2]",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4463,0,fde798ca3796eddd124c98a933f5d645e8b6e631835942...,4463,02,83,man,2020-08-29,6 days,NaN,0,[],...,3,0,NaN,None,False,3,Equipo atención primaria,"[3, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1]",0,NaN
4464,0,fe7f9d9090c393da6190b7a67fde053d62d4aeccd5009c...,4464,02,43,man,2020-03-23,7 days,100,2,"[OTROS TIPOS DE NEUMONIA VIRICA, OTROS CORONAV...",...,1,0,NaN,None,False,3,Equipo atención primaria,"[3, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1]",0,NaN
4465,0,fec24c409cd89a246ed9bcf040fb6dfa64d4f0df2a70bc...,4465,02,69,man,2020-08-28,21 days,100,0,[],...,0,0,NaN,None,False,3,Consultas externas,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]",0,NaN


In [56]:
df_experiment.drop(columns = ['_p_id']).reset_index().to_csv('table_extended_PCR.csv', index = False)


In [47]:
df_experiment.to_csv('table_extended_PCR.csv')

In [ ]:
df[df._p_id ==  4450]._patient_id

In [ ]:
df_experiment.date_icu_in.value_counts()

In [ ]:
df_experiment.iloc[0].covid_rel_image

In [ ]:
df[(df._p_id == 4467) & (df._event_type == 'Lab Sars-cov-2')]

In [ ]:


import pandas as pd
admissions_df = df_dict['State Admission']
discharge_df = df_dict['State Discharge']


In [ ]:
admissions_df.loc[admissions_df._p_id == 3308]

In [ ]:
p = 3308
#df_admissions 

In [ ]:
list(zip(admissions_df.loc[admissions_df._p_id == p]._start_date, 
         discharge_df.loc[discharge_df._p_id == p]._start_date,
         discharge_df.loc[discharge_df._p_id == p]._health_dep,
        admissions_df.loc[admissions_df._p_id == p]._p_id,
        admissions_df.loc[admissions_df._p_id == p]._gender,
        admissions_df.loc[admissions_df._p_id == p]._age,
        discharge_df.loc[discharge_df._p_id == p]._event_value))

In [ ]:
discharge_df['date'] = discharge_df['_start_date'] 
discharge_df

In [ ]:
m = pd.merge(admissions_df,discharge_df[['_p_id', 'date']], sort = True)
m.loc[m._p_id == 3308]._start_date.values, m.loc[m._p_id == 3308].date.values

In [ ]:
m.loc[m._p_id == 3308]._health_dep.unique()

In [ ]:
m.loc[m._p_id == 3308]._patient_id.unique()

In [ ]:
pd.merge(admissions_df,discharge_df, how='outer',on='_p_id')

In [ ]:
df_out = df

In [ ]:
def f(x):
    df.groupby('_p_id')
    
    return i

df_out['_out_discharge'] = df_out.apply(lambda x: f(x), axis = 1)

In [ ]:
DT = dt.Frame(df)

In [ ]:
DT.names

In [ ]:
DT[:,{'count':count()} , by('_p_id')]

In [ ]:
DT[1,classes_field_dict['Patient'] , by('_p_id')]

In [ ]:
df._patient_id.value_counts()

In [ ]:
#profile = ProfileReport(df,  pool_size = 32)
df._dose.describe()

In [ ]:
DT[ (f._event_value == 'POTASIO CLORURO') ,fields_fluid]

In [ ]:
dt.unique(DT[:, f._health_dep])

In [ ]:
DT[(f.event_type == 'Text Discharge') ,:]

In [ ]:
e = DT[(f.event_type == 'Text Discharge'), :][:,{'N': dt.count()} , by('_event_value')]

In [ ]:
e.to_csv(str(csv_dir/'Text_Discharge.csv'))

In [ ]:
DT[(dt.isna(f.date) == False),[f.date, f._start_date]]

In [ ]:
DT[dt.rowany(dt.isna(f.date) == False),[f.date, f._start_date]]

In [ ]:
DT.names

In [ ]:
pdict['08_01d7a0442ab14089134c009df5aafdc1a8d206197e27db555281cf6582fda9fd'].__dict__.keys()

In [ ]:
DT.names

In [ ]:
DT[(dt.isna(f.event_value) == True) & (dt.isna(f._event_value) == False),['event_type', '_event_value', 'event_value']]

In [ ]:
DT[dt.rowany(dt.isna(f.event_value) == False),[f.event_value, f._event_value, f._desc]]

In [ ]:
DT[((f.event_type == 'Medication') & dt.isna(f._dose) == True  & (f._health_dep == '08')), :]